<a id='toc'></a>


![readmebanner](https://i.pinimg.com/originals/80/0d/58/800d58c4377ac582c1174125dd83a4f4.png)
![exesummary]()

___

## Workflow

1. [Import](#import)
1. [Acquire Data](#acquire-data)
1. [Clean, Prep & Split Data](#clean-prep-and-split-data)
1. [Explore Data](#explore-data)
    - [Hypothesis Testing](#hypothesis-testing)
1. [Feature Importance](#feature-importance)
1. [Modeling](#modeling)
    - [Identify Baseline](#identify-baseline)
    - [Model on Train](#model-on-train)
    - [Model on Validate](#model-on-validate)
    - [Model on Test](#model-on-test)
  
7. [Drivers](#drivers)

8. [Conclusion](#conclusion)


___
# <a name="import"></a>1. Import 
Import all necessary libraries and functions. 

In [1]:
# acquire
import json
from typing import Dict, List, Optional, Union, cast
import requests
import numpy as np
import pandas as pd
from env import github_token, github_username
from requests import get
from bs4 import BeautifulSoup
import os

import acquire_zach as az
import wrangle as w


# prepare
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# explore
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image
from wordcloud import ImageColorGenerator

# model
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from model import get_metrics

 [Jump to Table of Contents](#toc)
___
# <a name="acquire-data"></a>2. Acquire Data
#### Reads in  `google_readmes1020.csv` from your local file using the `get_aac()` function in [`wrangle.py`](wrangle.py)

 [Jump to Table of Contents](#toc)
___
# <a name="clean-prep-and-split-data"></a>3. Clean, Prep and Split Data
Using two functions located in `wrangle.py` file:
- [`clean_aac()`](wrangle.py)
- [`split_df()`](wrangle.py)

I will perform the cleaning, prepping and splitting of my data:

 [Jump to Table of Contents](#toc)
___
# <a name="explore-data"></a>4. Explore Data

 [Jump to Table of Contents](#toc)
___

# <a name="hypothesis-testing"></a>• Hypothesis Testing

 [Jump to Table of Contents](#toc)
___
# <a name="feature-importance"></a>5. Feature Importance

 [Jump to Table of Contents](#toc)
___
# <a name="modeling"></a>6. Modeling

 [Jump to Table of Contents](#toc)
___
# <a name="identify-baseline"></a>• Identify Baseline

 [Jump to Table of Contents](#toc)
___
# <a name="model-on-train"></a>• Model on Train

[Jump to Table of Contents](#toc)
___
# <a name="model-on-validate"></a>• Model on Validate

[Jump to Table of Contents](#toc)
___
# <a name="model-on-test"></a>• Model on Test

[Jump to Table of Contents](#toc)
___
# <a name="drivers"></a>12. Drivers

[Jump to Table of Contents](#toc)
___
# <a name="conclusion"></a>13. Conclusion

[Jump to Table of Contents](#toc)

<a id='bottom'></a>